In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Preprocessing

----
## ***Optimization - Drop "STATUS" Column as Well***
----


In [ ]:
# Drop the non-beneficial ID columns, 'EIN', 'STATUS', and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME","STATUS"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = []
app_types_counts = application_df["APPLICATION_TYPE"].value_counts()
app_types_names = app_types_counts.index
n = 0

for app_type_count in app_types_counts:
  if app_type_count<=10:
    app_type = app_types_names[n]
    application_types_to_replace.append(app_type)
  n +=1

print(application_types_to_replace)

['T25', 'T14', 'T29', 'T15', 'T17']


In [ ]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_value_counts = application_df["CLASSIFICATION"].value_counts()
class_value_names = class_value_counts.index
n=0

for classification_count in class_value_counts:
  if classification_count>1:
    print(f'{class_value_names[n]} - {classification_count}')
  n+=1

C1000 - 17326
C2000 - 6074
C1200 - 4837
C3000 - 1918
C2100 - 1883
C7000 - 777
C1700 - 287
C4000 - 194
C5000 - 116
C1270 - 114
C2700 - 104
C2800 - 95
C7100 - 75
C1300 - 58
C1280 - 50
C1230 - 36
C1400 - 34
C7200 - 32
C2300 - 32
C1240 - 30
C8000 - 20
C7120 - 18
C1500 - 16
C1800 - 15
C6000 - 15
C1250 - 14
C8200 - 11
C1238 - 10
C1278 - 10
C1235 - 9
C1237 - 9
C7210 - 7
C2400 - 6
C1720 - 6
C4100 - 6
C1257 - 5
C1600 - 5
C1260 - 3
C2710 - 3
C0 - 3
C3200 - 2
C1234 - 2
C1246 - 2
C1267 - 2
C1256 - 2


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = []
class_types_counts = application_df["CLASSIFICATION"].value_counts()
class_types_names = class_types_counts.index
n = 0

for class_type_count in class_types_counts:
  if class_type_count<=10:
    class_type = class_types_names[n]
    classifications_to_replace.append(class_type)
  n +=1

print(classifications_to_replace)

['C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


In [ ]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
Other      118
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Check replacemnts worked for applications
application_df["APPLICATION_TYPE"].unique()

array(['T10', 'T3', 'T5', 'T7', 'T4', 'T6', 'T2', 'T9', 'T19', 'T8',
       'T13', 'T12', 'Other'], dtype=object)

In [ ]:
# Check replacements worked for categoreies
application_df["CLASSIFICATION"].unique()

array(['C1000', 'C2000', 'C3000', 'C1200', 'C2700', 'C7000', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C2100', 'Other',
       'C5000', 'C7120', 'C1800', 'C1400', 'C1270', 'C2300', 'C8200',
       'C1500', 'C1300', 'C1230', 'C1280', 'C1240', 'C1250', 'C8000'],
      dtype=object)

In [ ]:
# Display dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)
application_dummies_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1230,CLASSIFICATION_C1240,CLASSIFICATION_C1250,CLASSIFICATION_C1270,CLASSIFICATION_C1280,CLASSIFICATION_C1300,CLASSIFICATION_C1400,CLASSIFICATION_C1500,CLASSIFICATION_C1700,CLASSIFICATION_C1800,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2300,CLASSIFICATION_C2700,CLASSIFICATION_C2800,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C6000,CLASSIFICATION_C7000,CLASSIFICATION_C7100,CLASSIFICATION_C7120,CLASSIFICATION_C7200,CLASSIFICATION_C8000,CLASSIFICATION_C8200,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
features_array = application_dummies_df.drop(columns=["IS_SUCCESSFUL"]).values

target_array = (application_dummies_df["IS_SUCCESSFUL"]).values


In [ ]:
#Diplay our split target array
target_array

array([1, 1, 0, ..., 0, 1, 0])

In [ ]:
# Display our split features array
features_array

array([[    5000,        0,        1, ...,        0,        1,        0],
       [  108590,        0,        0, ...,        0,        1,        0],
       [    5000,        0,        0, ...,        0,        1,        0],
       ...,
       [    5000,        0,        0, ...,        0,        1,        0],
       [    5000,        0,        0, ...,        0,        1,        0],
       [36500179,        0,        0, ...,        0,        1,        0]])

In [ ]:
# Create a StandardScaler Instance
scaler = StandardScaler()

# Fit the StandardScaler
features_scaler = scaler.fit(features_array)

# Scale the Features
scaled_features = features_scaler.transform(features_array)

----
## ***Optimization-Using PCA***
----

In [ ]:
# Create PCA Model
pca_model=PCA(n_components=0.95)

In [ ]:
#Fit PCA Model
pca_features_array = pca_model.fit_transform(scaled_features)
pca_features_array

array([[ 3.29066748e+00,  9.76201876e-01,  1.87456696e-01, ...,
        -1.86094575e-01, -2.44672808e-01,  1.38866683e+00],
       [-1.16149161e+00,  1.98396605e+00,  2.59873326e-01, ...,
         1.75855136e+00,  6.66552253e-02, -7.41703788e-01],
       [ 6.16766631e+00,  6.43063080e-01,  3.96572430e-02, ...,
         1.54451290e+00, -4.56564572e-01,  1.45508724e-01],
       ...,
       [ 8.84909937e-01, -2.49214850e+00,  1.29061695e-02, ...,
        -1.33332860e-03,  3.11445674e-02,  4.48422114e-02],
       [ 5.17648280e+00,  2.16456467e+00,  5.60713996e-02, ...,
         1.46527535e+00, -6.83654028e-01,  1.19160159e+00],
       [-1.29244858e+00,  2.54084408e+00, -1.96122978e-01, ...,
        -5.95893298e-01,  4.03241853e-01, -3.45858365e-01]])

In [ ]:
# Check Model's Variance Ratio
var_ratio = pca_model.explained_variance_ratio_
var_ratio

array([0.06952299, 0.04136316, 0.02948821, 0.02763268, 0.02527966,
       0.02317433, 0.02229061, 0.02104798, 0.01909743, 0.01726591,
       0.01709078, 0.01683283, 0.01649766, 0.01620277, 0.01601861,
       0.01590955, 0.01558519, 0.0154447 , 0.01530938, 0.01522869,
       0.01509598, 0.01506664, 0.01500875, 0.01493237, 0.01489459,
       0.01480117, 0.01478857, 0.0147556 , 0.01474991, 0.01473923,
       0.01473512, 0.01472803, 0.01472272, 0.01471976, 0.01471849,
       0.01471515, 0.01471345, 0.01470928, 0.01469855, 0.01464848,
       0.01460655, 0.01458482, 0.01457911, 0.01450492, 0.01448359,
       0.01443614, 0.01425424, 0.01404007, 0.01397234, 0.01357633,
       0.01327919, 0.01319289, 0.01310985, 0.01237758, 0.01091678])

In [ ]:
# Check Total of Variance Ratios
tot_explained = sum(var_ratio)
tot_explained

0.9581393723544779

----
## ***Optimization-Using PCA Complete***
----

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(pca_features_array ,target_array,random_state=1)

## Compile, Train and Evaluate the Model

# Optimized Model Attempt 1-Same Setup as Original, but With PCA, and without "STATUS" Column
-----

In [ ]:
# Count features
features_count = len(X_train[0])
features_count


55

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units=4, activation="relu", input_dim=features_count))

# Second layer
nn.add(tf.keras.layers.Dense(units=2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_89 (Dense)            (None, 4)                 224       
                                                                 
 dense_90 (Dense)            (None, 2)                 10        
                                                                 
 dense_91 (Dense)            (None, 1)                 3         
                                                                 
Total params: 237 (948.00 Byte)
Trainable params: 237 (948.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
# Train the model
fit_nn = nn.fit(X_train, y_train, epochs=100)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.6905 - accuracy: 0.6574
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5985 - accuracy: 0.7125
Epoch 3/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5795 - accuracy: 0.7203
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5717 - accuracy: 0.7252
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5673 - accuracy: 0.7263
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5648 - accuracy: 0.7260
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5630 - accuracy: 0.7262
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5624 - accuracy: 0.7261
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5617 - accuracy: 0.7260
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5613 - accuracy: 0.7262

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6060 - accuracy: 0.6854 - 473ms/epoch - 2ms/step
Loss: 0.6059646010398865, Accuracy: 0.6853644251823425


# Optimized Model Attempt 2-With More Neruons
-----

In [ ]:
# Count features
features_count = len(X_train[0])
features_count

55

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units=110, activation="relu", input_dim=features_count))

# Second layer
nn.add(tf.keras.layers.Dense(units=55, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_113 (Dense)           (None, 110)               6160      
                                                                 
 dense_114 (Dense)           (None, 55)                6105      
                                                                 
 dense_115 (Dense)           (None, 1)                 56        
                                                                 
Total params: 12321 (48.13 KB)
Trainable params: 12321 (48.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
# Train the model
fit_nn = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5752 - accuracy: 0.7183
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5516 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5503 - accuracy: 0.7310
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7331
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5442 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5677 - accuracy: 0.7275 - 480ms/epoch - 2ms/step
Loss: 0.5676690936088562, Accuracy: 0.7274635434150696


# Optimized Model Attempt 3-Adding a Hidden Layer
-----

In [ ]:
# Count features
features_count = len(X_train[0])
features_count

55

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units=110, activation="relu", input_dim=features_count))

# Second layer
nn.add(tf.keras.layers.Dense(units=55, activation="relu"))

# Third layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()
nn.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_116 (Dense)           (None, 110)               6160      
                                                                 
 dense_117 (Dense)           (None, 55)                6105      
                                                                 
 dense_118 (Dense)           (None, 25)                1400      
                                                                 
 dense_119 (Dense)           (None, 1)                 26        
                                                                 
Total params: 13691 (53.48 KB)
Trainable params: 13691 (53.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5541 - accuracy: 0.7280
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5523 - accuracy: 0.7277
Epoch 3/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5500 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7335
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5465 - accuracy: 0.7344
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7340
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7340
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5581 - accuracy: 0.7294 - 375ms/epoch - 1ms/step
Loss: 0.5581441521644592, Accuracy: 0.7294460535049438


# Export Best Optimized Model
-----

In [361]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [362]:
# Export our model to HDF5 file
nn.save("/content/drive/MyDrive/Colab Notebooks/AlphabetSoupCharity_Optimization.HDF5")